In [1]:
import numpy as np
import torch

In [2]:
from models import MPS, ComplexTensor

In [3]:
from models import build_ghz_plus

In [4]:
from tools import generate_binary_space

In [5]:
from qtools import pauli_exp

In [6]:
L=2
local_dim=2
bond_dim=1

In [7]:
basis = torch.tensor(generate_binary_space(L), dtype=torch.long)

In [8]:
# psi = MPS(L=L, local_dim=local_dim, bond_dim=bond_dim)
psi = build_ghz_plus(L)

In [9]:
psi.shape

[(1, 2), (2, 1)]

In [10]:
psi.norm_full()

tensor(1.0000, grad_fn=<SqueezeBackward0>)

In [13]:
for i in range(len(basis)):
    print("basis state {0} has amplitude {1}".format(basis[i],
                                                    psi.amplitude_normalized()))

tensor([[1, 1],
        [1, 0],
        [0, 1],
        [0, 0]])